In [ ]:
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading
import numpy as np
import pandas as pd

import subprocess
import time
import signal
from typing import Dict

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Success, Failure, Task, TaskDispatcher


from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

from returns.pipeline import is_successful
from returns.result import Failure

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.capture.tshark import StartCapture, StopCapture
#from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

from netunicorn.base import DockerImage

from scapy.all import *
import pandas as pd
import numpy as np
import binascii
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline
pd.set_option('display.max_rows', None)

# Load Your Data

In [ ]:
twitch_csvs = ['data/twitch/path_to_your_csv']

# Convert Data Types and Filter for Flows that Contain the Local IP

In [ ]:
import ipaddress

# Function to convert IP to integer
def ip_to_int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return None

youtube_packets = pd.DataFrame()
for v in youtube_csvs:
    # Read in CSV
    curr_df = pd.read_csv(v,
                          sep='\t',
                          header=None,
                          dtype={
                              0: 'float',
                              1: 'string',
                              2: 'Int64',
                              3: 'Int64',
                              4: 'string',
                              5: 'Int64',
                              6: 'Int64',
                              7: 'string',
                              8: 'string',
                              9: 'string',
                              10: 'string',
                              11: 'Int64',
                              12: 'Int64',
                              13: 'Int64',
                              14: 'Int64',
                              15: 'Int64',
                              16: 'float',
                              17: 'Int64',
                              18: 'Int64'
                          })
    
    curr_df = curr_df.rename(columns={
        0: 'frame_timestamp',
        1: 'ip_src',
        2: 'tcp_srcport',
        3: 'udp_srcport',
        4: 'ip_dst',
        5: 'tcp_dstport',
        6: 'udp_dstport',
        7: 'ip_len',
        8: 'ip_header_len',
        9: 'protocol',
        10: 'tcp_flags',
        11: 'tcp_seq_num',
        12: 'tcp_ack_num',
        13: 'tcp_header_len',
        14: 'udp_len',
        15: 'tcp_retransmission',
        16: 'rtt',
        17: 'seq',
        18: 'ack'
    })
    
    # Combine src and dst ports
    curr_df['src_port'] = curr_df['tcp_srcport'].combine_first(curr_df['udp_srcport'])
    curr_df['dst_port'] = curr_df['tcp_dstport'].combine_first(curr_df['udp_dstport'])

    # Drop original TCP and UDP port columns
    curr_df = curr_df.drop(columns=['tcp_srcport', 'udp_srcport', 'tcp_dstport', 'udp_dstport'])

    # Get basename of file
    basename = v.split('/')[-1].split('.')[0]
    curr_df['file'] = basename
    
    # Convert IPs to integers
    curr_df['ip_src_int'] = curr_df['ip_src'].apply(ip_to_int)
    curr_df['ip_dst_int'] = curr_df['ip_dst'].apply(ip_to_int)
    
    curr_df = curr_df.drop(columns=['ip_src', 'ip_dst']).rename(columns={
        'ip_src_int': 'ip_src',
        'ip_dst_int': 'ip_dst'
    })
    
    # Append packets to current DF
    ttv = pd.concat([youtube_packets, curr_df], ignore_index=True)

In [ ]:
ttv.head()

In [ ]:
# Function to convert IP to integer
def ip_to_int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return None

# Use the IP's below if you are using the data we provided. Use the 172.... address for the netUnicorn data and the 52... address for the self collected _data. Otherwise, use your own!

In [ ]:
all_packets = ttv.copy()
client_ip = ip_to_int("YOUR IP")
#User the IP's below if you are using the data we provided. Use the 172.... address for the netUnicorn data and thee 52... address for the self collected _data. 
#client_ip = ip_to_int("172.17.0.2")
#client_ip = ip_to_int("52.223.247.239")
client_condition = (all_packets['ip_src'] == client_ip) | (all_packets['ip_dst'] == client_ip)

all_packets = all_packets[client_condition]

# Setting the correct data type
all_packets["protocol"] = all_packets["protocol"].astype(int)
all_packets['udp_len'] = all_packets['udp_len'].fillna(0).astype(int)
all_packets['ip_header_len'] = all_packets['ip_header_len'].fillna(0).astype(int)
all_packets['ip_len'] = all_packets['ip_len'].fillna(0).astype(int)
all_packets['tcp_header_len'] = all_packets['tcp_header_len'].fillna(0).astype(int)
all_packets['rtt'] = all_packets['rtt'].fillna(0).astype(float)
all_packets['tcp_retransmission'] = all_packets['tcp_retransmission'].fillna(0).astype(int)

In [ ]:
all_packets.head()

# Filter for TCP Traffic

In [ ]:
tcp_protocol_condition = all_packets['protocol'] == 6
tcp_packets = all_packets[tcp_protocol_condition].copy()



tcp_packets['tcp_len'] = tcp_packets['ip_len'].astype('Int64') - (tcp_packets['ip_header_len'].astype('Int64') + tcp_packets['tcp_header_len'].astype('Int64'))
tcp_packets['udp_len'] = 0  # No UDP length for TCP packets

tcp_packets = tcp_packets.sort_index()

In [ ]:
test = tcp_packets.copy()

# As Twitch uses HLS to stream video, organize data into chunks based on 4 Mbps - 8 Mbps estimates

In [ ]:
import random 

current_chunk = 1
threshold_mbps = random.randint(4,8)
current_data_size = 0
start_timestamp = test.iloc[0]['frame_timestamp']

for i, row in test.iterrows():
    frame_timestamp = row['frame_timestamp']
    ip_len = row['ip_len']
    
    # Calculate the time difference in seconds between the current frame and the previous frame
    time_diff = (frame_timestamp - start_timestamp) / 1e9  # Convert to seconds
    
    if time_diff > 0:  # Prevent division by zero
        data_rate = (ip_len * 8) / (time_diff * 1e6)  # Calculate Mbps (Mbps = (Bytes * 8) / time_seconds / 10^6)
    else:
        data_rate = 0
    
    # Add the current row's size to the chunk
    current_data_size += ip_len * 8  # In bits
    
    # If the data rate threshold is reached, create a new chunk
    if current_data_size >= threshold_mbps * 1e6:  # Compare in bits
        threshold_mbps = random.randint(4, 8)  # Pick a new random threshold
        current_data_size = 0  # Reset the data size for the next chunk
        current_chunk += 1  # Increment chunk number
        
    test.at[i, 'chunk'] = current_chunk
    start_timestamp = frame_timestamp 
    

# Group data based on Chunk.

In [ ]:
test_grouped = test.groupby('chunk').agg(
    total_tcp_len=('tcp_len', 'sum'),
    rtt=('rtt', 'sum'),
    dropped_packets=('tcp_retransmission', 'sum'),
    time_chunk_sent=('frame_timestamp', 'min'),
    download_duration=('frame_timestamp', lambda x: x.max() - x.min())
).reset_index()


In [ ]:
test_grouped.head()

In [ ]:
print(test_grouped['total_tcp_len'].mean())
print(test_grouped['total_tcp_len'].std())

In [ ]:
test_grouped['rtt'] = test_grouped['rtt'] * 1000
test_grouped['throughput'] = ((test_grouped['total_tcp_len'] * 8) / test_grouped['download_duration']) / 1000000
test_grouped['bytes_per_transmission_time'] = test_grouped['total_tcp_len'] / test_grouped['rtt']

In [ ]:
test_grouped.head()

# Create a CSV of your Data

In [ ]:
test_grouped.to_csv('data/test_data/your_data.csv', index=False)